In [1]:
import json

In [2]:
from pyspark.sql import SparkSession

In [10]:
config = json.load(open("config.json"))

In [4]:
spark = SparkSession.builder\
    .config("spark.jars", config['JDBC_PATH']) \
    .config("spark.executor.extraClassPath", config['JDBC_PATH'])\
    .getOrCreate()

24/01/09 00:45:14 WARN Utils: Your hostname, przemyslaw-Latitude-E7450 resolves to a loopback address: 127.0.1.1; using 192.168.1.110 instead (on interface wlp2s0)
24/01/09 00:45:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/01/09 00:45:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/09 00:45:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/01/09 00:45:17 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [5]:
dbtable = "customer"

In [15]:
df_customer = spark.read.format("jdbc")\
        .option("url",config['SRC_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", dbtable)\
        .option("user", config['SRC_DB_USER'])\
        .option("password", config['SRC_DB_PASSWORD'])\
        .load()

In [16]:
df_customer.createOrReplaceTempView(dbtable)

In [17]:
dbtable = "address"

In [18]:
df_address = spark.read.format("jdbc")\
        .option("url",config['SRC_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", dbtable)\
        .option("user", config['SRC_DB_USER'])\
        .option("password", config['SRC_DB_PASSWORD'])\
        .load()

In [19]:
df_address.createOrReplaceTempView(dbtable)

In [20]:
dbtable = "city"

In [21]:
df_city = spark.read.format("jdbc")\
        .option("url",config['SRC_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", dbtable)\
        .option("user", config['SRC_DB_USER'])\
        .option("password", config['SRC_DB_PASSWORD'])\
        .load()

In [22]:
df_city.createOrReplaceTempView(dbtable)

In [23]:
dbtable = "country"

In [24]:
df_country = spark.read.format("jdbc")\
        .option("url",config['SRC_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", dbtable)\
        .option("user", config['SRC_DB_USER'])\
        .option("password", config['SRC_DB_PASSWORD'])\
        .load()

In [25]:
df_country.createOrReplaceTempView(dbtable)

In [26]:
dim_customer = spark.sql("""
SELECT
ROW_NUMBER() OVER(ORDER BY c.customer_id) AS customer_id,
c.customer_id AS customer_key,
CONCAT(c.first_name, " ", c.last_name) AS customer_name,
c.email AS customer_email,
c.birth_date AS customer_birth_date,
a.address AS customer_address,
a.postal_code AS customer_postal_code,
ci.city AS customer_city,
co.country AS customer_country
FROM customer AS c
JOIN address AS a ON c.address_id = a.address_id
JOIN city AS ci ON a.city_id = ci.city_id
JOIN country co ON ci.country_id = co.country_id 
""")

In [27]:
dim_customer.show()

24/01/09 00:49:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:49:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:49:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:49:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:49:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:49:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 0

+-----------+------------+--------------------+--------------------+-------------------+--------------------+--------------------+----------------+----------------+
|customer_id|customer_key|       customer_name|      customer_email|customer_birth_date|    customer_address|customer_postal_code|   customer_city|customer_country|
+-----------+------------+--------------------+--------------------+-------------------+--------------------+--------------------+----------------+----------------+
|          1|          25|  Przemysław Drobnik|przemyslaw.drobni...|         1972-11-11|       Kosynierów 03|              71-304|           Konin|          Poland|
|          2|          32|    Zdenka Svobodová|zdenka.svobodova@...|         1964-08-20|  Viale Ruggiero 729|               15074|         Fiastra|           Italy|
|          3|          33|Thibaut-Denis Lec...|thibaut-denis.lec...|         1974-05-06|   Pyörökiventie 497|               14507|   Uusikaarlepyy|         Finland|
|         

In [28]:
dim_customer.write.format("jdbc")\
        .option("url",config['DST_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", "dim_customer")\
        .option("user", config['DST_DB_USER'])\
        .option("password", config['DST_DB_PASSWORD'])\
        .save()

24/01/09 00:51:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:51:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:51:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:51:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:51:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 00:51:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 0